In [1]:
import time
import sys
import os
import config
import config_runtime
import subprocess
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [2]:
# — Asegurar UTF-8 en stdout —
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8", errors="replace")

def safe_print(text):
    """Imprime sin romper con caracteres Unicode."""
    try:
        print(text)
    except UnicodeEncodeError:
        enc = sys.stdout.encoding or "utf-8"
        fixed = text.encode(enc, errors="replace").decode(enc)
        print(fixed)

def ejecutar_script(path):
    """Lanza un script Python y captura stdout/stderr."""
    safe_print(f"\n🚀 Ejecutando: {path}")
    env = os.environ.copy()
    env["PYTHONIOENCODING"] = "utf-8"
    result = subprocess.run(
        ["python", path],
        env=env,
        capture_output=True,
        text=True
    )
    if result.returncode == 0:
        safe_print("✅ Script OK")
    else:
        safe_print(f"⚠️ Exit code {result.returncode}")
    safe_print("🔹 STDOUT:")
    safe_print(result.stdout)
    safe_print("🔸 STDERR:")
    safe_print(result.stderr)

def save_config_runtime():
    lines = [
        f"usuario = {config_runtime.usuario!r}\n",
        f"reporte = {config_runtime.reporte!r}\n",
        f"pbit_queue = {config_runtime.pbit_queue!r}\n",
    ]
    with open("config_runtime.py", "w", encoding="utf-8") as f:
        f.writelines(lines)

class PbitHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory or not event.src_path.lower().endswith(".pbit"):
            return
        config_runtime.pbit_queue.append(event.src_path)
        save_config_runtime()
        print(f"📑 Cola actualizada: {config_runtime.pbit_queue}")

In [ ]:
# Celda 1: Detectar y encolar .pbit
watch_folder = config.base_report_directory
observer = Observer()
observer.schedule(PbitHandler(), path=watch_folder, recursive=False)
observer.start()
print(f"👀 Observando carpeta: {watch_folder} (Ctrl+C para parar)")
try:
    while True: time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()


👀 Observando carpeta: C:\Users\Administrator\DMV Automation - 10 Jun\Input - Reportes (Ctrl+C para parar)
📑 Cola actualizada: ['C:\\Users\\Administrator\\DMV Automation - 10 Jun\\Input - Reportes\\testUser + Empower BI V3 - Copy (2).pbit']
📑 Cola actualizada: ['C:\\Users\\Administrator\\DMV Automation - 10 Jun\\Input - Reportes\\testUser + Empower BI V3 - Copy (2).pbit', 'C:\\Users\\Administrator\\DMV Automation - 10 Jun\\Input - Reportes\\testUser + Empower BI V3 - Copy - Copy - Copy - Copy.pbit']
📑 Cola actualizada: ['C:\\Users\\Administrator\\DMV Automation - 10 Jun\\Input - Reportes\\testUser + Empower BI V3 - Copy (2).pbit', 'C:\\Users\\Administrator\\DMV Automation - 10 Jun\\Input - Reportes\\testUser + Empower BI V3 - Copy - Copy - Copy - Copy.pbit', 'C:\\Users\\Administrator\\DMV Automation - 10 Jun\\Input - Reportes\\testUser + Empower BI V3 - Copy - Copy - Copy (2).pbit']
📑 Cola actualizada: ['C:\\Users\\Administrator\\DMV Automation - 10 Jun\\Input - Reportes\\testUser + Emp

In [ ]:

# Celda 2: Para cada .pbit en cola → ejecutar Script 1 y Two_A

for pbit in config_runtime.pbit_queue:
    # Actualizar reporte
    config_runtime.reporte = os.path.basename(pbit)
    # Ejecutar extracción de DMVs (Script 1)
    ejecutar_script(config.script_one_path)
    # Ejecutar análisis de dependencias (Two_A)
    ejecutar_script(config.script_two_a_path)



🚀 Ejecutando: C:\Users\Administrator\DMV Automation - 10 Jun\One_Extraccion_DMVs_V3.py


In [ ]:
# Celda 3: Ejecutar consolidación (Two_B) UNA vez
from main import ejecutar_script
import config

ejecutar_script(config.script_two_b_path)


In [ ]:
# Celda 4: Ejecutar publicación (Three) UNA vez
from main import ejecutar_script
import config

ejecutar_script(config.script_three_path)


In [1]:
import config_runtime
for pbit in config_runtime.pbit_queue:
    print(pbit)

C:\Users\Administrator\DMV Automation - 10 Jun\Input - Reportes\testUser + Empower BI V3 - Copy (2).pbit
C:\Users\Administrator\DMV Automation - 10 Jun\Input - Reportes\testUser + Empower BI V3 - Copy - Copy - Copy - Copy.pbit
C:\Users\Administrator\DMV Automation - 10 Jun\Input - Reportes\testUser + Empower BI V3 - Copy - Copy - Copy (2).pbit
C:\Users\Administrator\DMV Automation - 10 Jun\Input - Reportes\testUser + Empower BI V3 - Copy - Copy (2).pbit
